In [6]:
import pandas as pd

# Load the Excel files
file1 = r"C:\Users\incha\Downloads\Results\FPLM_loop_FVA_cobra.xlsx"  # Contains 'reactions', 'minimum', 'maximum'
file2 = r"C:\Users\incha\Downloads\Results\sort_keggbiggmatchcorefunc.xlsx"  # Contains 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway'

df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

# --- Step 1: Explode reactions in File 1 ---
df1['reactions'] = df1['reactions'].astype(str)
df1_exploded = df1.assign(reaction_bit=df1['reactions'].str.split('_')).explode('reaction_bit')
df1_exploded['reaction_bit'] = df1_exploded['reaction_bit'].str.strip()

# --- Step 2: Explode BiGG_IDs in File 2 ---
df2['BiGG_IDs'] = df2['BiGG_IDs'].astype(str)
df2_exploded = df2.assign(BiGG_ID_split=df2['BiGG_IDs'].str.split(',')).explode('BiGG_ID_split')
df2_exploded['BiGG_ID_split'] = df2_exploded['BiGG_ID_split'].str.strip()

# --- Step 3: Match reactions to BiGG_IDs ---
matched_df = df1_exploded.merge(df2_exploded, left_on='reaction_bit', right_on='BiGG_ID_split', how='inner')
matched_df = matched_df[['reactions', 'minimum', 'maximum', 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway']].drop_duplicates()
matched_df.to_excel(r"C:\Users\incha\Downloads\Results\FPLM_Core_FVA_match.xlsx", index=False)

# --- Step 4: Find unmatched reactions from File 1 ---
all_combinations = df1[['reactions', 'minimum', 'maximum']].drop_duplicates().merge(
    matched_df[['reactions']], on='reactions', how='left', indicator=True
)
unmatched_reactions = all_combinations[all_combinations['_merge'] == 'left_only'].drop(columns=['_merge'])
unmatched_reactions['Core Function'] = pd.NA
unmatched_reactions['KeggId'] = pd.NA
unmatched_reactions['BiGG_IDs'] = pd.NA
unmatched_reactions['Pathway'] = pd.NA
unmatched_reactions = unmatched_reactions[['reactions', 'minimum', 'maximum', 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway']]
unmatched_reactions.to_excel(r"C:\Users\incha\Downloads\Results\FPLM_Core_FVA_unmatch.xlsx", index=False)

# --- Step 5: Find unmatched BiGG_IDs from File 2 (grouped form) ---
# Get matched individual BiGG_IDs
matched_bigg_ids = set(matched_df['BiGG_IDs'].dropna().unique())

# Re-explode BiGG_IDs to check individually
df2_exploded_unique = df2.assign(BiGG_ID_split=df2['BiGG_IDs'].str.split(',')).explode('BiGG_ID_split')
df2_exploded_unique['BiGG_ID_split'] = df2_exploded_unique['BiGG_ID_split'].str.strip()
df2_exploded_unique['is_matched'] = df2_exploded_unique['BiGG_ID_split'].isin(matched_bigg_ids)

# Keep only groups where no BiGG_IDs matched
unmatched_bigg_rows = df2_exploded_unique.groupby(['Core Function', 'KeggId', 'Pathway']).filter(
    lambda group: not group['is_matched'].any()
)

# Group back to original BiGG_IDs structure (comma-separated)
unmatched_bigg_grouped = unmatched_bigg_rows.groupby(['Core Function', 'KeggId', 'Pathway'])['BiGG_ID_split'] \
    .apply(lambda x: ', '.join(sorted(set(x)))).reset_index().rename(columns={'BiGG_ID_split': 'BiGG_IDs'})

# Save to Excel
unmatched_bigg_grouped.to_excel(r"C:\Users\incha\Downloads\Results\Core_unmatch_FPLM.xlsx", index=False)


In [7]:
import pandas as pd

# Load the Excel files
file1 = r"C:\Users\incha\Downloads\Results\FLM_loop_FVA_cobra.xlsx"  # Contains 'reactions', 'minimum', 'maximum'
file2 = r"C:\Users\incha\Downloads\Results\sort_keggbiggmatchcorefunc.xlsx"  # Contains 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway'

df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

# --- Step 1: Explode reactions in File 1 ---
df1['reactions'] = df1['reactions'].astype(str)
df1_exploded = df1.assign(reaction_bit=df1['reactions'].str.split('_')).explode('reaction_bit')
df1_exploded['reaction_bit'] = df1_exploded['reaction_bit'].str.strip()

# --- Step 2: Explode BiGG_IDs in File 2 ---
df2['BiGG_IDs'] = df2['BiGG_IDs'].astype(str)
df2_exploded = df2.assign(BiGG_ID_split=df2['BiGG_IDs'].str.split(',')).explode('BiGG_ID_split')
df2_exploded['BiGG_ID_split'] = df2_exploded['BiGG_ID_split'].str.strip()

# --- Step 3: Match reactions to BiGG_IDs ---
matched_df = df1_exploded.merge(df2_exploded, left_on='reaction_bit', right_on='BiGG_ID_split', how='inner')
matched_df = matched_df[['reactions', 'minimum', 'maximum', 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway']].drop_duplicates()
matched_df.to_excel(r"C:\Users\incha\Downloads\Results\FLM_Core_FVA_match.xlsx", index=False)

# --- Step 4: Find unmatched reactions from File 1 ---
all_combinations = df1[['reactions', 'minimum', 'maximum']].drop_duplicates().merge(
    matched_df[['reactions']], on='reactions', how='left', indicator=True
)
unmatched_reactions = all_combinations[all_combinations['_merge'] == 'left_only'].drop(columns=['_merge'])
unmatched_reactions['Core Function'] = pd.NA
unmatched_reactions['KeggId'] = pd.NA
unmatched_reactions['BiGG_IDs'] = pd.NA
unmatched_reactions['Pathway'] = pd.NA
unmatched_reactions = unmatched_reactions[['reactions', 'minimum', 'maximum', 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway']]
unmatched_reactions.to_excel(r"C:\Users\incha\Downloads\Results\FLM_Core_FVA_unmatch.xlsx", index=False)

# --- Step 5: Find unmatched BiGG_IDs from File 2 (grouped form) ---
# Get matched individual BiGG_IDs
matched_bigg_ids = set(matched_df['BiGG_IDs'].dropna().unique())

# Re-explode BiGG_IDs to check individually
df2_exploded_unique = df2.assign(BiGG_ID_split=df2['BiGG_IDs'].str.split(',')).explode('BiGG_ID_split')
df2_exploded_unique['BiGG_ID_split'] = df2_exploded_unique['BiGG_ID_split'].str.strip()
df2_exploded_unique['is_matched'] = df2_exploded_unique['BiGG_ID_split'].isin(matched_bigg_ids)

# Keep only groups where no BiGG_IDs matched
unmatched_bigg_rows = df2_exploded_unique.groupby(['Core Function', 'KeggId', 'Pathway']).filter(
    lambda group: not group['is_matched'].any()
)

# Group back to original BiGG_IDs structure (comma-separated)
unmatched_bigg_grouped = unmatched_bigg_rows.groupby(['Core Function', 'KeggId', 'Pathway'])['BiGG_ID_split'] \
    .apply(lambda x: ', '.join(sorted(set(x)))).reset_index().rename(columns={'BiGG_ID_split': 'BiGG_IDs'})

# Save to Excel
unmatched_bigg_grouped.to_excel(r"C:\Users\incha\Downloads\Results\Core_unmatch_FLM.xlsx", index=False)


In [8]:
import pandas as pd

# Load the Excel files
file1 = r"C:\Users\incha\Downloads\Results\FPL_loop_FVA_cobra.xlsx"  # Contains 'reactions', 'minimum', 'maximum'
file2 = r"C:\Users\incha\Downloads\Results\sort_keggbiggmatchcorefunc.xlsx"  # Contains 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway'

df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

# --- Step 1: Explode reactions in File 1 ---
df1['reactions'] = df1['reactions'].astype(str)
df1_exploded = df1.assign(reaction_bit=df1['reactions'].str.split('_')).explode('reaction_bit')
df1_exploded['reaction_bit'] = df1_exploded['reaction_bit'].str.strip()

# --- Step 2: Explode BiGG_IDs in File 2 ---
df2['BiGG_IDs'] = df2['BiGG_IDs'].astype(str)
df2_exploded = df2.assign(BiGG_ID_split=df2['BiGG_IDs'].str.split(',')).explode('BiGG_ID_split')
df2_exploded['BiGG_ID_split'] = df2_exploded['BiGG_ID_split'].str.strip()

# --- Step 3: Match reactions to BiGG_IDs ---
matched_df = df1_exploded.merge(df2_exploded, left_on='reaction_bit', right_on='BiGG_ID_split', how='inner')
matched_df = matched_df[['reactions', 'minimum', 'maximum', 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway']].drop_duplicates()
matched_df.to_excel(r"C:\Users\incha\Downloads\Results\FPL_Core_FVA_match.xlsx", index=False)

# --- Step 4: Find unmatched reactions from File 1 ---
all_combinations = df1[['reactions', 'minimum', 'maximum']].drop_duplicates().merge(
    matched_df[['reactions']], on='reactions', how='left', indicator=True
)
unmatched_reactions = all_combinations[all_combinations['_merge'] == 'left_only'].drop(columns=['_merge'])
unmatched_reactions['Core Function'] = pd.NA
unmatched_reactions['KeggId'] = pd.NA
unmatched_reactions['BiGG_IDs'] = pd.NA
unmatched_reactions['Pathway'] = pd.NA
unmatched_reactions = unmatched_reactions[['reactions', 'minimum', 'maximum', 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway']]
unmatched_reactions.to_excel(r"C:\Users\incha\Downloads\Results\FPL_Core_FVA_unmatch.xlsx", index=False)

# --- Step 5: Find unmatched BiGG_IDs from File 2 (grouped form) ---
# Get matched individual BiGG_IDs
matched_bigg_ids = set(matched_df['BiGG_IDs'].dropna().unique())

# Re-explode BiGG_IDs to check individually
df2_exploded_unique = df2.assign(BiGG_ID_split=df2['BiGG_IDs'].str.split(',')).explode('BiGG_ID_split')
df2_exploded_unique['BiGG_ID_split'] = df2_exploded_unique['BiGG_ID_split'].str.strip()
df2_exploded_unique['is_matched'] = df2_exploded_unique['BiGG_ID_split'].isin(matched_bigg_ids)

# Keep only groups where no BiGG_IDs matched
unmatched_bigg_rows = df2_exploded_unique.groupby(['Core Function', 'KeggId', 'Pathway']).filter(
    lambda group: not group['is_matched'].any()
)

# Group back to original BiGG_IDs structure (comma-separated)
unmatched_bigg_grouped = unmatched_bigg_rows.groupby(['Core Function', 'KeggId', 'Pathway'])['BiGG_ID_split'] \
    .apply(lambda x: ', '.join(sorted(set(x)))).reset_index().rename(columns={'BiGG_ID_split': 'BiGG_IDs'})

# Save to Excel
unmatched_bigg_grouped.to_excel(r"C:\Users\incha\Downloads\Results\Core_unmatch_FPL.xlsx", index=False)


In [9]:
import pandas as pd

# Load the Excel files
file1 = r"C:\Users\incha\Downloads\Results\FPM_loop_FVA_cobra.xlsx"  # Contains 'reactions', 'minimum', 'maximum'
file2 = r"C:\Users\incha\Downloads\Results\sort_keggbiggmatchcorefunc.xlsx"  # Contains 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway'

df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

# --- Step 1: Explode reactions in File 1 ---
df1['reactions'] = df1['reactions'].astype(str)
df1_exploded = df1.assign(reaction_bit=df1['reactions'].str.split('_')).explode('reaction_bit')
df1_exploded['reaction_bit'] = df1_exploded['reaction_bit'].str.strip()

# --- Step 2: Explode BiGG_IDs in File 2 ---
df2['BiGG_IDs'] = df2['BiGG_IDs'].astype(str)
df2_exploded = df2.assign(BiGG_ID_split=df2['BiGG_IDs'].str.split(',')).explode('BiGG_ID_split')
df2_exploded['BiGG_ID_split'] = df2_exploded['BiGG_ID_split'].str.strip()

# --- Step 3: Match reactions to BiGG_IDs ---
matched_df = df1_exploded.merge(df2_exploded, left_on='reaction_bit', right_on='BiGG_ID_split', how='inner')
matched_df = matched_df[['reactions', 'minimum', 'maximum', 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway']].drop_duplicates()
matched_df.to_excel(r"C:\Users\incha\Downloads\Results\FPM_Core_FVA_match.xlsx", index=False)

# --- Step 4: Find unmatched reactions from File 1 ---
all_combinations = df1[['reactions', 'minimum', 'maximum']].drop_duplicates().merge(
    matched_df[['reactions']], on='reactions', how='left', indicator=True
)
unmatched_reactions = all_combinations[all_combinations['_merge'] == 'left_only'].drop(columns=['_merge'])
unmatched_reactions['Core Function'] = pd.NA
unmatched_reactions['KeggId'] = pd.NA
unmatched_reactions['BiGG_IDs'] = pd.NA
unmatched_reactions['Pathway'] = pd.NA
unmatched_reactions = unmatched_reactions[['reactions', 'minimum', 'maximum', 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway']]
unmatched_reactions.to_excel(r"C:\Users\incha\Downloads\Results\FPM_Core_FVA_unmatch.xlsx", index=False)

# --- Step 5: Find unmatched BiGG_IDs from File 2 (grouped form) ---
# Get matched individual BiGG_IDs
matched_bigg_ids = set(matched_df['BiGG_IDs'].dropna().unique())

# Re-explode BiGG_IDs to check individually
df2_exploded_unique = df2.assign(BiGG_ID_split=df2['BiGG_IDs'].str.split(',')).explode('BiGG_ID_split')
df2_exploded_unique['BiGG_ID_split'] = df2_exploded_unique['BiGG_ID_split'].str.strip()
df2_exploded_unique['is_matched'] = df2_exploded_unique['BiGG_ID_split'].isin(matched_bigg_ids)

# Keep only groups where no BiGG_IDs matched
unmatched_bigg_rows = df2_exploded_unique.groupby(['Core Function', 'KeggId', 'Pathway']).filter(
    lambda group: not group['is_matched'].any()
)

# Group back to original BiGG_IDs structure (comma-separated)
unmatched_bigg_grouped = unmatched_bigg_rows.groupby(['Core Function', 'KeggId', 'Pathway'])['BiGG_ID_split'] \
    .apply(lambda x: ', '.join(sorted(set(x)))).reset_index().rename(columns={'BiGG_ID_split': 'BiGG_IDs'})

# Save to Excel
unmatched_bigg_grouped.to_excel(r"C:\Users\incha\Downloads\Results\Core_unmatch_FPM.xlsx", index=False)


In [10]:
import pandas as pd

# Load the Excel files
file1 = r"C:\Users\incha\Downloads\Results\PLM_loop_FVA_cobra.xlsx"  # Contains 'reactions', 'minimum', 'maximum'
file2 = r"C:\Users\incha\Downloads\Results\sort_keggbiggmatchcorefunc.xlsx"  # Contains 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway'

df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

# --- Step 1: Explode reactions in File 1 ---
df1['reactions'] = df1['reactions'].astype(str)
df1_exploded = df1.assign(reaction_bit=df1['reactions'].str.split('_')).explode('reaction_bit')
df1_exploded['reaction_bit'] = df1_exploded['reaction_bit'].str.strip()

# --- Step 2: Explode BiGG_IDs in File 2 ---
df2['BiGG_IDs'] = df2['BiGG_IDs'].astype(str)
df2_exploded = df2.assign(BiGG_ID_split=df2['BiGG_IDs'].str.split(',')).explode('BiGG_ID_split')
df2_exploded['BiGG_ID_split'] = df2_exploded['BiGG_ID_split'].str.strip()

# --- Step 3: Match reactions to BiGG_IDs ---
matched_df = df1_exploded.merge(df2_exploded, left_on='reaction_bit', right_on='BiGG_ID_split', how='inner')
matched_df = matched_df[['reactions', 'minimum', 'maximum', 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway']].drop_duplicates()
matched_df.to_excel(r"C:\Users\incha\Downloads\Results\PLM_Core_FVA_match.xlsx", index=False)

# --- Step 4: Find unmatched reactions from File 1 ---
all_combinations = df1[['reactions', 'minimum', 'maximum']].drop_duplicates().merge(
    matched_df[['reactions']], on='reactions', how='left', indicator=True
)
unmatched_reactions = all_combinations[all_combinations['_merge'] == 'left_only'].drop(columns=['_merge'])
unmatched_reactions['Core Function'] = pd.NA
unmatched_reactions['KeggId'] = pd.NA
unmatched_reactions['BiGG_IDs'] = pd.NA
unmatched_reactions['Pathway'] = pd.NA
unmatched_reactions = unmatched_reactions[['reactions', 'minimum', 'maximum', 'Core Function', 'KeggId', 'BiGG_IDs', 'Pathway']]
unmatched_reactions.to_excel(r"C:\Users\incha\Downloads\Results\PLM_Core_FVA_unmatch.xlsx", index=False)

# --- Step 5: Find unmatched BiGG_IDs from File 2 (grouped form) ---
# Get matched individual BiGG_IDs
matched_bigg_ids = set(matched_df['BiGG_IDs'].dropna().unique())

# Re-explode BiGG_IDs to check individually
df2_exploded_unique = df2.assign(BiGG_ID_split=df2['BiGG_IDs'].str.split(',')).explode('BiGG_ID_split')
df2_exploded_unique['BiGG_ID_split'] = df2_exploded_unique['BiGG_ID_split'].str.strip()
df2_exploded_unique['is_matched'] = df2_exploded_unique['BiGG_ID_split'].isin(matched_bigg_ids)

# Keep only groups where no BiGG_IDs matched
unmatched_bigg_rows = df2_exploded_unique.groupby(['Core Function', 'KeggId', 'Pathway']).filter(
    lambda group: not group['is_matched'].any()
)

# Group back to original BiGG_IDs structure (comma-separated)
unmatched_bigg_grouped = unmatched_bigg_rows.groupby(['Core Function', 'KeggId', 'Pathway'])['BiGG_ID_split'] \
    .apply(lambda x: ', '.join(sorted(set(x)))).reset_index().rename(columns={'BiGG_ID_split': 'BiGG_IDs'})

# Save to Excel
unmatched_bigg_grouped.to_excel(r"C:\Users\incha\Downloads\Results\Core_unmatch_PLM.xlsx", index=False)
